Importing

In [2]:
#python 3.10.x need for this project
import numpy as np 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

c:\Users\hisha\Desktop\New\ISTOP\1\tfenv310\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Getting IMDB reviews

In [3]:
train_data, validation_data, test_data = tfds.load( name='imdb_reviews',
                                                   split=('train[:60%]', 'train[:60%]', 'test'),
                                                   as_supervised=True ) 

In [4]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [5]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [6]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

Embedding

In [8]:
embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(
    embedding_url,
    input_shape=[],          
    dtype=tf.string,
    trainable=True
)

In [9]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

Model

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=["accuracy"])


In [13]:
history = model.fit(train_data.shuffle(10000).batch(100),
                    epochs = 25,
                    validation_data = validation_data.batch(100),
                    verbose = 1
                    )

Epoch 1/25



150/150 [==============================] - 5s 25ms/step - loss: 0.6145 - accuracy: 0.6470 - val_loss: 0.5253 - val_accuracy: 0.7297
Epoch 2/25
150/150 [==============================] - 3s 21ms/step - loss: 0.4778 - accuracy: 0.7597 - val_loss: 0.4195 - val_accuracy: 0.8077
Epoch 3/25
150/150 [==============================] - 3s 19ms/step - loss: 0.3829 - accuracy: 0.8225 - val_loss: 0.3343 - val_accuracy: 0.8506
Epoch 4/25
150/150 [==============================] - 3s 19ms/step - loss: 0.3122 - accuracy: 0.8645 - val_loss: 0.2817 - val_accuracy: 0.8680
Epoch 5/25
150/150 [==============================] - 3s 19ms/step - loss: 0.2635 - accuracy: 0.8897 - val_loss: 0.2349 - val_accuracy: 0.9114
Epoch 6/25
150/150 [==============================] - 3s 19ms/step - loss: 0.2255 - accuracy: 0.9102 - val_loss: 0.1998 - val_accuracy: 0.9223
Epoch 7/25
150/150 [==============================] - 3s 19ms/step - loss: 0.1960 - accuracy: 0.9241 - val_loss: 0.1727 - val_accuracy: 0.9379
Epoch 8/25

In [14]:
results = model.evaluate(test_data.batch(100), verbose = 2)

for name, values in zip(model.metrics_names,results):
    print("%s : %.3f " %(name, values))

250/250 - 2s - loss: 0.6297 - accuracy: 0.8405 - 2s/epoch - 8ms/step
loss : 0.630 
accuracy : 0.841 


Test Cases 

In [15]:
def predict_review(review_text):
    prediction = model.predict([review_text])[0][0]
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    print(f"Review: {review_text}")
    print(f"Predicted Sentiment: {sentiment} (Confidence: {prediction:.2f})")

In [18]:
predict_review("This movie was a complete masterpiece!")

1/1 [==============================] - 0s 52ms/step
Review: This movie was a complete masterpiece!
Predicted Sentiment: Positive (Confidence: 2.62)


In [17]:
model.save('sentiment_model.h5')
loaded_model = tf.keras.models.load_model('sentiment_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})


c:\Users\hisha\Desktop\New\ISTOP\1\tfenv310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
